# Тестовое задание: Графовые нейронные сети, классические алгоритмы

## Настройка окружения

```
!conda install -c dglteam dgl
```

_Примечание 1_. Для установки пакета с возможностью обучения моделей на видеокартах необходимо также иметь установленный CUDA.

_Примечание 2_. dgl поддерживает работу с тремя фреймворками: PyTorch 1.5.0+, Apache MXNet 1.6+ и TensorFlow 2.3+. Мы работаем с PyTorch. Полный набор команд для развертывания окружения с возможностью обучения на GPU (cuda версии 10.2):

```cmd
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
python -c "import torch; print(torch.__version__)"
python -c "import torch; print(torch.version.cuda)"
conda install -c dglteam dgl-cuda10.2
python -m dgl.backend.set_default_backend pytorch
python -c "import dgl; print(dgl.__version__)"
```

DGL (Deep Graph Library)  - это пакет, разработанный для обучения и использования моделей графовых нейронных сетей. В данном пакете реализовано большое количество ставших уже классическими модулей (например, GraphConv, GraphSage и т.д.), на основе которых можно разрабатывать собственные модели. DGL предоставляет большой набор возможностей: работа с GPU, возможности для работы с большими графами и т.д.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np

import dgl
import dgl.nn as gnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Работа с графами в `dgl`

В `dgl` граф - объект класс `DGLGraph`. Для его создания нужно указать:
* тензор начальных узлов для ребер
* тензор конечных узлов для ребер
* кол-во вершин (узлы нумеруются, начиная с 0; кол-во вершин можно не задавать, если они все перечислены в списках для создания ребер)

В `dgl` граф всегда ориентированный.

На узлах (`.ndata`) и ребрах (`.edata`) могут храниться атрибуты. Их особенности:
* только числовые тензоры;
* атрибуты всех узлов (ребер) должны иметь одинаковый размер;
* каждый атрибут имеет уникальное имя (атрибуты узлов и ребер могут иметь одинаковые имена);
* первая размерность тензора атрибутов должна быть равна кол-ву узлов (ребер);
* срез по строкам возвращает атрибуты одного узла (ребра)

Пример создания графа:
```python
G = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
G.ndata['x'] = torch.randn(6, 3)
G.ndata['y'] = torch.randint(0, 2, (6, ))
```

Кроме возможностей по созданию наборов данных, `dgl` предоставляет большой набор готовых датасетов в `dgl.data`.

`DGLGraph` имеют большое количество свойств и методов для работы с графовыми структурами, которые можно найти в [документации](https://docs.dgl.ai/api/python/dgl.DGLGraph.html)

Задание 1. Загрузите датасет `CoraGraphDataset` из `dgl.data`. Этот датасет состоит из одного графа. Выведите на экран:

* количество узлов в графе;
* количество ребер в графе;
* размерность атрибутов на узлах;
* количество классов узлов в датасете

Выделите подграф (`dgl.node_subgraph`), содержащий узлы, относящиеся к трем наиболее часто встречающимся классам. __1 балл__


Задание 2. Файл `cities_nodes.csv` содержит данные о городах: название, страна, население, дату основания и координаты. В файле `cities_edges.csv` содержатся данные о связях между городам: города соединяются ребрами в случае, если в статье русскоязычной Википедии о городе присутсвует ссылка на страницу для другого города. Создайте на основе двух этих файлов `dgl.graph`. Для этого:


1. занумеруйте города (узлы) целыми числами;

2. представьте каждое ребро в виде пары двух целых чисел;

3. в качестве атрибутов (`g.ndata['feat']`) узлов используйте логарифм численности населения в городе и входящую степень узлов (результат сохраните в виде двумерного тензора размера `(n_nodes, 2)` и приведите к `torch.float32`);

4. занумеруйте страны целыми числами и в качестве меток узлов (`g.ndata['label']`) используйте номер соотвествующей страны; используйте тензор типа `torch.int64`;

5. в качестве атрибутов (`g.edata['distance']`) ребер используйте логарифм расстояния между городами (результат сохраните в виде одномерного тензора размера `(n_edges, )` и приведите к `torch.float32`);

6. добавьте графу атрибут `num_classes`, в котором хранится число классов (стран);

7. разбейте множество узлов на обучающее и валидационное множество в соотношении 90%-10%. Для представления каждого из множеств создайте булев тензор $t$ размерности, равной количеству узлов в графе, где $t[i]==True$, если узел $i$ входит в соответствующее множество. Добавьте в словарь `ndata` два ключа `train_mask`, `val_mask`. __(1 балл)__


## Построение нейронных сетей с использованием `dgl`

`dgl` предоставляет [большое количество](https://docs.dgl.ai/api/python/nn.html) модулей-блоков для построения нейронных сетей. С точки зрения реализации они являются наследниками стандартного `torch.nn.Module`. Поэтому общая логика написания сети остается точно такой же, как и при использовании `torch`: вы описываете модель при помощи класса-наследника `torch.nn.Module`, определяете в методе `__init__` все необходимые блоки и реализуете метод `forward`, где вызываете модули в нужной последовательности. Кроме модулей графовых сетей вы можете использовать и "классические": `Linear`, `Dropout` и т.д.

В отличие от модулей из `torch`, где метод `forward` обычно ожидает один параметр на входе (как `Linear` ожидает один тензор), модули из `dgl` обычно ожидают 2 параметра: граф и матрицу атрибутов узлов. Конкретный модуль может ожидать и чего-то другого, поэтому рекомендуется сверяться с документацией.

Пример работы с модулями из `dgl`

```python
import dgl.nn as gnn
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
g = dgl.add_reverse_edges(g)
g.ndata['x'] = torch.randn(6, 3)

conv = gnn.GraphConv(3, 32)
h = conv(g, g.ndata['x'])
h.shape # h - тензор скрытых представлений узлов после свертки conv
```

Пример описания графовой нейронной сети при помощи `dgl`
```python
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
```

Задание 3. Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу классификации городов по странам (задача классификации узлов). __(2 балла)__

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи
2. Значение функции потерь на обучающем множестве
3. Значение метрики accuracy (`sklearn.metrics.accuracy_score`) на обучающем множестве
3. Значение метрики accuracy (`sklearn.metrics.accuracy_score`) на тестовом множестве

Важно: при расчете значения функции потерь при обучении используйте только узлы, относящиеся к обучающему множеству! Узлы, относящиеся к валидационному множеству, для простоты можно оставить в графе, но прогнозы для этих узлов не должны оказывать влияние на функцию потерь и обратное распространение ошибки при обучении.

Задание 4. Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу предсказания связей. __(2 балла)__

Цикл обучения должен состоять из следующих шагов:
1. получение представлений `h` узлов при помощи сети, аналогичной заданию 3;
2. конструирование графа отрицательных примеров (см функцию `construct_negative_graph`);
3. расчет прогнозов ребер на основе исходного графа и скрытых представлений узлов `h` (см класс `MLPPredictor`) - для этих ребер модель должна предсказывать метку 1;
4. расчет прогнозов ребер на основе графа отрицательных примеров и и скрытых представлений узлов `h` (см класс `MLPPredictor`) - для этих ребер модель должна предсказывать метку 0;
5. расчет функции потерь (например, `CrossEntropyLoss`) и шаг оптимизации.

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи;
2. Значение функции потерь на обучающем множестве.

Примечания: 
1. для простоты в данной задаче не предполагается разбиение ребер на обучающее и тестовое множество;
2. на каждой эпохе получение эмбеддингов `h` происходит только 1 раз на основе исходного графа;

In [2]:
def construct_negative_graph(g, k):
    '''Берет ребра из графа g и случайным образом заменяет вершины на концах ребер'''
    u, v = g.edges()
    neg_u = u.repeat_interleave(k).long()
    neg_v = torch.randint(0, g.num_nodes(), (len(neg_u),)).long()
    return dgl.graph((neg_u, neg_v), num_nodes=g.num_nodes())

In [3]:
class MLPPredictor(nn.Module):
    '''Берет представления узлов, находящихся на концах ребер, конкатенирует их и прогоняет через небольшую полносвязную
    сеть для получения прогноза вероятности существования ребра'''
    def __init__(self, h_feats, out_feats=2):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, out_feats)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

Задание 5. Используя граф, полученный в задании 2, и графовую нейронную сеть, написанную с использованием библиотеки `dgl`, решите задачу предсказания расстояния между городами (задача регрессии на ребрах). __(2 балла)__

Цикл обучения должен состоять из следующих шагов:
1. получение представлений `h` узлов при помощи сети, аналогичной заданию 3;
3. расчет прогнозов ребер на основе исходного графа и скрытых представлений узлов `h` (см класс `MLPPredictor`);
5. расчет функции потерь (например, `MSELoss`) и шаг оптимизации.

На каждой эпохи обучения выводите на экран следующую информацию:
1. Номер эпохи
2. Значение функции потерь на обучающем множестве.

Примечание: для простоты в данной задаче не предполагается разбиение ребер на обучающее и тестовое множество.

Задание 6. Сведите эмбеддинги узлов, полученные при обучении модели в задаче 5, к векторам размерности 2, используя метод главных компонент (`sklearn.decomposition.PCA`). Визуализируйте сеть, используя теперь для координат точек векторное представление узлов размерности 2. Покажите цветом узла страну, к которой относится город. __(1 балл)__ 

Для преобразования `DGLGraph` к `nx.Graph` можно воспользоваться функцией `dgl.to_networkx`.